WiscLand2 is land cover data for the state of Wisconsin. I use this data to mask out anything but forests in my project.  This could be easily be adjusted, though, in the code below.

This notebook crops the Wiscland data and the images to a shapefile (such as your study area) and then masks the images of everything but "forest."  Reprojection and resampling are done automatically.

About WiscLand2:
https://www.sco.wisc.edu/2016/09/23/wiscland-2-project-complete-data-now-available/
Download Wiscland2 (will automatically download!):
https://data-wi-dnr.opendata.arcgis.com/datasets/d7f5d33b182044c187c776e47d72ce84

In [7]:
import os
from osgeo import gdal, gdal_array
import numpy as np
import numpy.ma as ma
import rasterio

In [9]:
### This is the only block you need to edit. ###

# Location of my image files and where output will be saved
os.chdir('.../files')

# Image names in the filepath given above
image_fps = ['20190924_164332_56_1058_3B_AnalyticMS_SR_clip_masked.tif',
            '20190926_164553_77_1061_3B_AnalyticMS_SR_clip_masked.tif']

# Location of the wiscland data
wiscland = '.../WiscLand2/wiscland2_landcover/wiscland2/wiscland2_dataset/level1/wiscland2_level1.tif'

# Location of shape (study area, AOI) to crop the images and Wiscland to
shape = '...AOI.shp'

# Type of imagery. This is used for resampling purposes.
sensor = 'Dove'

In [10]:
def crop_plot(shape, org_img, crop_file):
    ''' 
    Crops orignal imagery to extent of desired shape.
    Input a shapefile with ONE feature
    Shape = shapefile to be used for cropping
    Org_img = original imagery to be cropped
    crop_file = File for cropped imagery to be written to
    ''' 
    
    import fiona 
    import rasterio 
    import rasterio.mask 
    
    with fiona.open(shape, 'r') as shapefile:
        features = [feature['geometry'] for feature in shapefile]
    
    with rasterio.open(org_img) as src:
        out_image, out_transform = rasterio.mask.mask(src, features, crop=True)
        out_meta = src.meta.copy()
        
    out_meta.update({"driver": "GTiff",
                     "height": out_image.shape[1],
                     "width": out_image.shape[2],
                     "transform": out_transform,
                     "nodata":0})
    
    with rasterio.open(crop_file, "w", **out_meta) as dest:
        dest.write(out_image)
        
    shape=None

In [11]:
if sensor == 'Dove':
    res = 3
elif sensor == 'Sentinel-2':
    res = 10
elif sensor == 'Landsat' or sensor == 'HLS':
    res = 30    

3


In [12]:
# Reproject mask (as needed) and align spatial resolution. Crop mask to image bounds.
for image in image_fps:
    print(image)
    raster = rasterio.open(image)
    wl = rasterio.open(wiscland)
    img_crs = raster.crs
    profile = wl.profile
    wl_crs = profile['crs']
    limits = [raster.bounds[0], raster.bounds[1], raster.bounds[2], raster.bounds[3]]
    # Generate gdal warp options 
    warp_options = gdal.WarpOptions(format='GTiff', 
                                    outputBounds=limits,
                                    xRes=res,                  
                                    yRes=res,
                                    srcSRS=wl_crs,
                                    dstSRS=img_crs,
                                    resampleAlg='near',
                                    multithread=True)
    
    wl_reproj = '%s_Reprojected_wiscland.tif' %image[:-4]
    
    rerun = gdal.Warp(wl_reproj, wiscland, options=warp_options)
    rerun = None
    # Check the profile for extent and spatial resolution 
    wl_rep = rasterio.open(wl_reproj)

    meta = wl_rep.meta
    print(meta)
    wiscland = wl_reproj
    
    image_clip_fp = '%s_clip.tif' %image[:-4]
    wl_clip_fp = 'Wiscland_clip.tif'
    
    crop_plot(shape, image, image_clip_fp)
    crop_plot(shape, wiscland, wl_clip_fp)
    
    #Create mask
    mask_file = gdal.Open(wl_clip_fp)
    mask_data = mask_file.GetRasterBand(1)
    mask_array = gdal_array.BandReadAsArray(mask_data)

    # forest pixels are 4000
    mask = (mask_array == 4000)
    qa_mask = np.invert(mask) #Inverstion is necessary to work with ma.masked_array

    # Mask bands 
    arr = np.zeros((np.shape(qa_mask)[0], np.shape(qa_mask)[1]),  dtype=np.uint16)  # pre-allocate

    org_image = gdal.Open(image_clip_fp)
    num_bands = range(1, org_image.RasterCount+1)

    masked_fp = '%s_masked.tif' %image_clip_fp[:-4]

    for band in num_bands:
        print('Masking band %s' %band)
        band_data = org_image.GetRasterBand(band)
        band_array = gdal_array.BandReadAsArray(band_data)

        tmp = ma.masked_array(band_array, qa_mask)
        arr = ma.filled(tmp, 0)  # fill mask using nodata value.

        # Write data to a geotiff
        with rasterio.open(image_clip_fp) as src:
            kwds = src.profile
            kwds['nodata'] = 0
            kwds['driver'] = 'GTiff'
            #kwds['dtype'] = rasterio.int16
            kwds['width'] = arr.shape[1]
            kwds['height'] = arr.shape[0]
            kwds['count'] = org_image.RasterCount


            if band == 1:          
                with rasterio.open(masked_fp, 'w', **kwds) as dst:
                     dst.write(np.array(arr).astype(rasterio.uint16), band)
            else:
                with rasterio.open(masked_fp, 'r+', **kwds) as dst:
                     dst.write(np.array(arr).astype(rasterio.uint16), band)


20190924_164332_56_1058_3B_AnalyticMS_SR_clip_masked.tif
{'driver': 'GTiff', 'dtype': 'uint16', 'nodata': None, 'width': 3026, 'height': 1341, 'count': 1, 'crs': CRS.from_epsg(32616), 'transform': Affine(3.0, 0.0, 316920.0,
       0.0, -3.0, 5096574.0)}
Masking band 1
Masking band 2
Masking band 3
Masking band 4
20190926_164553_77_1061_3B_AnalyticMS_SR_clip_masked.tif
{'driver': 'GTiff', 'dtype': 'uint16', 'nodata': None, 'width': 3026, 'height': 1341, 'count': 1, 'crs': CRS.from_epsg(32616), 'transform': Affine(3.0, 0.0, 316920.0,
       0.0, -3.0, 5096574.0)}
Masking band 1
Masking band 2
Masking band 3
Masking band 4
